In [0]:
! git clone https://github.com/apache/spark.git

Cloning into 'spark'...
remote: Enumerating objects: 884764, done.
remote: Counting objects: 100% (1077/1077), done.
remote: Compressing objects: 100% (516/516), done.
remote: Total 884764 (delta 351), reused 891 (delta 288), pack-reused 883687
Receiving objects: 100% (884764/884764), 401.70 MiB | 10.01 MiB/s, done.
Resolving deltas: 100% (384181/384181), done.
Updating files: 100% (18236/18236), done.


In [0]:
ls /databricks/driver

conf/  eventlogs/  logs/  metastore_db/  preload_class.lst*  spark/


In [0]:
ls

conf/  eventlogs/  logs/  metastore_db/  preload_class.lst*  spark/


In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName(name='ML_Tutorial_Creation').getOrCreate()

In [0]:
# Binomial Logistic Regression
from pyspark.ml.classification import LogisticRegression

In [0]:
training = spark.read.format(source="libsvm").load("file:///databricks/driver/spark/data/mllib/sample_libsvm_data.txt")

In [0]:
training.show(truncate=False,n=5)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
lr=LogisticRegression(featuresCol="features", labelCol="label",maxIter=10,elasticNetParam=0.8)

lrModel=lr.fit(dataset=training)

In [0]:
print(lrModel.coefficientMatrix)

1 X 692 CSRMatrix
(0,95) -0.0013
(0,96) -0.0003
(0,97) 0.0034
(0,98) 0.0051
(0,99) 0.0023
(0,100) 0.0003
(0,101) -0.0039
(0,102) -0.0016
(0,119) 0.0047
(0,120) 0.0016
(0,121) 0.0023
(0,122) -0.0004
(0,123) 0.0001
(0,124) 0.0004
(0,125) 0.0004
(0,126) 0.0003
..
..


In [0]:
print(lrModel.intercept)

3.0451245076622677


In [0]:
# Multinomial Logistics Regression

mlr=LogisticRegression(featuresCol="features",labelCol="label",maxIter=10,elasticNetParam=0.8,family="multinomial")

mlrModel=mlr.fit(dataset=training)

In [0]:
print(mlrModel.coefficientMatrix)

2 X 692 CSRMatrix
(0,95) 0.0013
(0,96) 0.0003
(0,97) -0.0012
(0,98) -0.0021
(0,99) -0.0007
(0,100) 0.0003
(0,101) 0.0019
(0,102) 0.0001
(0,119) -0.0025
(0,120) -0.0008
(0,121) -0.0011
(0,122) 0.0004
(0,123) 0.0001
(0,124) -0.0
(0,125) -0.0001
(0,126) -0.0
..
..


In [0]:
print(mlrModel.interceptVector)

[-2.72615312578918,2.72615312578918]


In [0]:
# Get the Training Summary
trainingSummary=lrModel.summary

objectiveHistory=trainingSummary.objectiveHistory

print(objectiveHistory)

[0.6833149135741672, 0.013093751340219123, 0.010701411598307595, 0.003694038375993432, 0.002139976133815725, 0.0011363374603547175, 0.0006407228823852678, 0.0003670542855157364, 0.00022600264220548632, 0.0001509437536659315, 0.00010710631663632266]


In [0]:
  # Receiver Operator Characteristics
  trainingSummary.roc.show()

+---+--------------------+
|FPR|                 TPR|
+---+--------------------+
|0.0|                 0.0|
|0.0|0.017543859649122806|
|0.0| 0.03508771929824561|
|0.0| 0.05263157894736842|
|0.0| 0.07017543859649122|
|0.0| 0.08771929824561403|
|0.0| 0.10526315789473684|
|0.0| 0.12280701754385964|
|0.0| 0.14035087719298245|
|0.0| 0.15789473684210525|
|0.0| 0.17543859649122806|
|0.0| 0.19298245614035087|
|0.0| 0.21052631578947367|
|0.0| 0.22807017543859648|
|0.0| 0.24561403508771928|
|0.0|  0.2631578947368421|
|0.0|  0.2807017543859649|
|0.0|  0.2982456140350877|
|0.0|  0.3157894736842105|
|0.0|  0.3333333333333333|
+---+--------------------+
only showing top 20 rows

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [0]:
print(trainingSummary.areaUnderROC)

1.0


In [0]:
trainingSummary.fMeasureByThreshold.show(300)

/databricks/spark/python/pyspark/sql/context.py:134: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
+--------------------+--------------------+
|           threshold|           F-Measure|
+--------------------+--------------------+
|  0.9999985497923128|0.034482758620689655|
|  0.9999979962514823| 0.06779661016949151|
|  0.9999974837508653|                 0.1|
|  0.9999974388920254| 0.13114754098360656|
|   0.999996973155266| 0.16129032258064516|
|  0.9999964677287961|  0.1904761904761905|
|  0.9999960750090817|             0.21875|
|  0.9999950517518691| 0.24615384615384614|
|  0.9999883840314531|  0.2727272727272727|
|  0.9999878745085975| 0.29850746268656714|
|  0.9999818567132517|  0.3235294117647059|
|  0.9999761032094638| 0.34782608695652173|
|  0.9999753944048935|  0.3714285714285714|
|  0.9999751144701023|  0.3943661971830986|
|  0.9999746999974123|  0.4166666666666667|
|  0.9999736302538365|  0.4383561643835616|
|  0.99997

In [0]:
fMeasure=trainingSummary.fMeasureByThreshold

fMeasure.groupBy().max('F-Measure').select("max(F-Measure)").show()

Out[59]: Row(max(F-Measure)=1.0)

In [0]:
maxFMeasure=fMeasure.groupBy().max('F-Measure').select("max(F-Measure)").head()
maxFMeasure

Out[60]: Row(max(F-Measure)=1.0)

In [0]:
fMeasure.where(fMeasure['F-Measure']==maxFMeasure['max(F-Measure)']).show()

+------------------+---------+
|         threshold|F-Measure|
+------------------+---------+
|0.9993099582238999|      1.0|
+------------------+---------+



In [0]:
bestThreshold = fMeasure.where(fMeasure['F-Measure']==maxFMeasure['max(F-Measure)']).select('threshold').head()['threshold']
bestThreshold

Out[66]: 0.9993099582238999

In [0]:
lr.setThreshold(value=bestThreshold)

Out[67]: LogisticRegression_cd35e0dc8ae4

In [0]:
# Multinomial Logistic Regression - Continued

In [0]:
training=spark.read.format(source="libsvm").load("file:///databricks/driver/spark/data/mllib/sample_multiclass_classification_data.txt")
training.show(truncate=False, n=4)

+-----+-------------------------------------------------------+
|label|features                                               |
+-----+-------------------------------------------------------+
|1.0  |(4,[0,1,2,3],[-0.222222,0.5,-0.762712,-0.833333])      |
|1.0  |(4,[0,1,2,3],[-0.555556,0.25,-0.864407,-0.916667])     |
|1.0  |(4,[0,1,2,3],[-0.722222,-0.166667,-0.864407,-0.833333])|
|1.0  |(4,[0,1,2,3],[-0.722222,0.166667,-0.694915,-0.916667]) |
+-----+-------------------------------------------------------+
only showing top 4 rows



In [0]:
lr=LogisticRegression(featuresCol="features",labelCol="label",maxIter=10,regParam=0.3,elasticNetParam=0.8)

lrModel=lr.fit(dataset=training)

In [0]:
# Coeficients
print(lrModel.coefficientMatrix)

3 X 4 CSRMatrix
(0,3) 0.305
(1,2) -0.7666
(1,3) -0.3854


In [0]:
# Intercept
print(lrModel.interceptVector)

[0.051925800207288736,-0.12619173083599086,0.0742659306287021]


In [0]:
trainingSummary=lrModel.summary

In [0]:
# False Positive Rate by Label

trainingSummary.falsePositiveRateByLabel

Out[80]: [0.22, 0.05, 0.0]

In [0]:
# True Positive Rate by Label

trainingSummary.truePositiveRateByLabel

Out[81]: [1.0, 1.0, 0.46]

In [0]:
# Precision by Label

trainingSummary.precisionByLabel

Out[82]: [0.6944444444444444, 0.9090909090909091, 1.0]

In [0]:
# Recall by Label

trainingSummary.recallByLabel

Out[83]: [1.0, 1.0, 0.46]

In [0]:
# F-Measure by Label

trainingSummary.fMeasureByLabel()

Out[85]: [0.819672131147541, 0.9523809523809523, 0.6301369863013699]

In [0]:
accuracy=trainingSummary.accuracy
falsePositiveRate=trainingSummary.weightedFalsePositiveRate
truePositiveRate=trainingSummary.weightedTruePositiveRate
fMeasure=trainingSummary.weightedFMeasure()
precision=trainingSummary.weightedPrecision
recall=trainingSummary.weightedRecall

print("Accuracy: %s\nFPR: %s\nTPR: %s\nF-measure: %s\nPrecision: %s\nRecall: %s"
      % (accuracy, falsePositiveRate, truePositiveRate, fMeasure, precision, recall))

Accuracy: 0.82
FPR: 0.09
TPR: 0.82
F-measure: 0.800730023276621
Precision: 0.8678451178451179
Recall: 0.82


In [0]:
# Decision Tree Classifier
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import countDistinct 

In [0]:
data=spark.read.format(source="libsvm").load("file:///databricks/driver/spark/data/mllib/sample_libsvm_data.txt")

data.show(truncate=False,n=2)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
data.select(countDistinct('label')).show()

+---------------------+
|count(DISTINCT label)|
+---------------------+
|                    2|
+---------------------+



In [0]:
labelIndexer=StringIndexer(inputCol="label",outputCol="indexedLabel").fit(dataset=data)

featureIndexer=VectorIndexer(inputCol="features",outputCol="indexedFeatures", maxCategories=4).fit(dataset=data)

In [0]:
trainingData, testData=data.randomSplit(weights=(0.7,0.3,),seed=101)

dt=DecisionTreeClassifier(featuresCol="indexedFeatures", labelCol="indexedLabel")

In [0]:
pipeline=Pipeline(stages=(labelIndexer,featureIndexer,dt))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions=model.transform(dataset=testData)

predictions.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
predictions.select('indexedLabel','indexedFeatures','prediction').show(n=5,truncate=False)

+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="indexedLabel",metricName="accuracy")


accuracy=evaluator.evaluate(dataset=predictions)

print(f"Test Error: {1-accuracy}")

Test Error: 0.0


In [0]:
# Model Summary
model.stages[2]

Out[119]: DecisionTreeClassificationModel: uid=DecisionTreeClassifier_fbd5a20af2cc, depth=2, numNodes=5, numClasses=2, numFeatures=692

In [0]:
# Random Forest Classifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString

In [0]:
rf=RandomForestClassifier(featuresCol="indexedFeatures", labelCol="indexedLabel",numTrees=10)


In [0]:
labelIndexer.labels

Out[178]: ['1.0', '0.0']

In [0]:
labelConverter=IndexToString(inputCol="prediction",outputCol="predictedLabel",labels=labelIndexer.labels)


In [0]:
pipeline=Pipeline(stages=(labelIndexer,featureIndexer,rf,labelConverter,))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions = model.transform(testData)
predictions.show(n=3,truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
predictions.select("predictedLabel", "label", "features").show(truncate=False)

+--------------+-----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
evaluator=MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="indexedLabel",metricName="accuracy")


accuracy=evaluator.evaluate(dataset=predictions)

print(f"Test Error: {1-accuracy}")

Test Error: 0.0


In [0]:
rfModel=model.stages[2]
rfModel

Out[187]: RandomForestClassificationModel: uid=RandomForestClassifier_3ff86916abb7, numTrees=10, numClasses=2, numFeatures=692

In [0]:
# Gradient Boost Tree Classifier
from pyspark.ml.classification import GBTClassifier


In [0]:
gbt=GBTClassifier(labelCol="indexedLabel",featuresCol="indexedFeatures",maxIter=10)

In [0]:
pipeline=Pipeline(stages=(labelIndexer,featureIndexer,gbt,))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions=model.transform(dataset=testData)

In [0]:
evaluator=MulticlassClassificationEvaluator(labelCol="indexedLabel",predictionCol="prediction",metricName="accuracy")

In [0]:
accuracy=evaluator.evaluate(dataset=predictions)
print(f"Test Error: {1-accuracy}")

Test Error: 0.10344827586206895


In [0]:
gbtModel=model.stages[2]
print(gbtModel)

GBTClassificationModel: uid = GBTClassifier_2c71a2cfaa37, numTrees=10, numClasses=2, numFeatures=692


In [0]:
# Multilayer Perceptron Classifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from typing import List

In [0]:
layers:List=[4,5,4,3]
trainer=MultilayerPerceptronClassifier(maxIter=100,layers=layers,blockSize=128,seed=1234)

In [0]:
# pipeline=Pipeline(stages=(labelIndexer,featureIndexer,trainer,))
pipeline=Pipeline(stages=(labelIndexer,featureIndexer,gbt,))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions=model.transform(dataset=testData)

In [0]:
predictions.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
evaluator.evaluate(dataset=predictions)

In [0]:
data=spark.read.format(source="libsvm").load("file:///databricks/driver/spark/data/mllib/sample_multiclass_classification_data.txt")
trainingData,testdata=data.randomSplit(weights=(0.7,0.3),seed=101)

In [0]:
trainingData.show(truncate=False,n=2)

+-----+-----------------------------------------------------+
|label|features                                             |
+-----+-----------------------------------------------------+
|0.0  |(4,[0,1,2,3],[-0.666667,-0.583333,0.186441,0.333333])|
|0.0  |(4,[0,1,2,3],[-0.277778,-0.333333,0.322034,0.583333])|
+-----+-----------------------------------------------------+
only showing top 2 rows



In [0]:
layers:List=[4,5,4,3]
trainer=MultilayerPerceptronClassifier(maxIter=100,layers=layers,blockSize=128,seed=1234)

In [0]:
model=trainer.fit(dataset=trainingData)

In [0]:
result=model.transform(dataset=testData)
result = model.transform(test)

In [0]:
result.show(truncate=False, n=4)

+-----+----------------------------------------------------+------------------------------------------------------------+----------------------------------------------------------------+----------+
|label|features                                            |rawPrediction                                               |probability                                                     |prediction|
+-----+----------------------------------------------------+------------------------------------------------------------+----------------------------------------------------------------+----------+
|0.0  |(4,[0,1,2,3],[-0.166667,-0.416667,0.38983,0.5])     |[20.563021846146164,-25.376755248819688,1.7402215241246344] |[0.9999999933109924,1.1184287049192863E-20,6.689007505728463E-9]|0.0       |
|0.0  |(4,[0,1,2,3],[-0.166667,-0.333333,0.38983,0.916667])|[20.560188293342435,-25.37160175857221,1.739057831940901]   |[0.9999999932998134,1.127397412169848E-20,6.7001865469970105E-9]|0.0       |
|0.0  |(4,

In [0]:
evaluator=MulticlassClassificationEvaluator(metricName="accuracy")
accuracy=evaluator.evaluate(dataset=result)
print(f"Test Error: {1-accuracy}")

Test Error: 0.08333333333333337


In [0]:
# Linear Support Vector Machine
from pyspark.ml.classification import LinearSVC

In [0]:
trainingData.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
lsvc=LinearSVC(featuresCol="indexedFeatures",labelCol="indexedLabel")

pipeline=Pipeline(stages=(labelIndexer,featureIndexer,lsvc,))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions=model.transform(dataset=testData)

In [0]:
predictions.show(truncate=False)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
evaluator=MulticlassClassificationEvaluator(metricName="accuracy",predictionCol='prediction',labelCol='indexedLabel')

In [0]:
accuracy=evaluator.evaluate(predictions)
print(f"Test Error: {1-accuracy}")

Test Error: 0.033333333333333326


In [0]:
# NaiveBias

from pyspark.ml.classification import NaiveBayes

In [0]:
nb=NaiveBayes(featuresCol='indexedFeatures',labelCol='indexedLabel',modelType='multinomial')
pipeline=Pipeline(stages=(labelIndexer,featureIndexer,nb,))

model=pipeline.fit(dataset=trainingData)

In [0]:
results=model.transform(dataset=testData)

In [0]:
evaluator=MulticlassClassificationEvaluator(metricName="accuracy",predictionCol='prediction',labelCol='indexedLabel')
accuracy=evaluator.evaluate(results)
print(f"Test Error: {1-accuracy}")

Test Error: 0.0


In [0]:
# Factorization Machines Classifier

from pyspark.ml.classification import FMClassifier
from pyspark.ml.feature import MinMaxScaler

In [0]:
data=spark.read.format(source="libsvm").load("file:///databricks/driver/spark/data/mllib/sample_libsvm_data.txt")

In [0]:
trainingData,testData=data.randomSplit(weights=(.75,.25),seed=101)


In [0]:
labelIndexer=StringIndexer(inputCol='label',outputCol='indexedLabel').fit(data)

featureScaler=MinMaxScaler(inputCol='features',outputCol='scaledFeatures').fit(data)

In [0]:
fm=FMClassifier(featuresCol='scaledFeatures',labelCol='indexedLabel',stepSize=1e-3)


In [0]:
pipeline=Pipeline(stages=(labelIndexer,featureScaler,fm,))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
results=model.transform(dataset=testData)

In [0]:
results.show(truncate=False, n=4)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
evaluator = MulticlassClassificationEvaluator(labelCol="indexedLabel", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(results)

In [0]:
print(f"Test Error: {1-accuracy}")

Test Error: 0.03703703703703709


In [0]:
# Regression

In [0]:
# Linear Regression
from pyspark.ml.regression import LinearRegression

In [0]:
training=spark.read.format(source="libsvm").load("file:///databricks/driver/spark/data/mllib/sample_linear_regression_data.txt")

In [0]:
training.show(truncate=False, n=4)

+-------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label              |features                                                                                                                                                                                                                            |
+-------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|-9.490009878824548 |(10,[0,1,2,3,4,5,6,7,8,9],[0.4551273600657362,0.36644694351969087,-0.38256108933468047,-0.4458430198517267,0.33109790358914726,0.8067445293443565,-0.2624341731773887,-0.44850386111659524,-0.07269284838169332,0.5658035575800715

In [0]:
lr=LinearRegression(featuresCol="features",labelCol="label",maxIter=10,regParam=0.3,elasticNetParam=0.8)

lrModel=lr.fit(dataset=training)

In [0]:
print(lrModel.coefficients)

[0.0,0.32292516677405936,-0.3438548034562218,1.9156017023458414,0.05288058680386263,0.765962720459771,0.0,-0.15105392669186682,-0.21587930360904642,0.22025369188813426]


In [0]:
print(lrModel.intercept)

0.1598936844239736


In [0]:
trainingSummary=lrModel.summary

In [0]:
print(f"Total Number of Iterations: {trainingSummary.totalIterations}")

Total Number of Iterations: 6


In [0]:
print(f"Objective History: {trainingSummary.objectiveHistory}")

Objective History: [0.49999999999999994, 0.4967620357443381, 0.4936361664340463, 0.4936351537897608, 0.4936351214177871, 0.49363512062528014, 0.4936351206216114]


In [0]:
trainingSummary.residuals.show(truncate=False)

+--------------------+
|residuals           |
+--------------------+
|-9.889232683103197  |
|0.5533794340053554  |
|-5.204019455758823  |
|-20.566686715507508 |
|-9.4497405180564    |
|-6.909112502719486  |
|-10.00431602969873  |
|2.062397807050484   |
|3.1117508432954772  |
|-15.893608229419382 |
|-5.036284254673026  |
|6.483215876994333   |
|12.429497299109002  |
|-20.32003219007654  |
|-2.0049838218725005 |
|-17.867901734183793 |
|7.646455887420495   |
|-2.2653482182417406 |
|-0.10308920436195645|
|-1.380034070385301  |
+--------------------+
only showing top 20 rows



In [0]:
print(f"RMSE: {trainingSummary.rootMeanSquaredError}")

RMSE: 10.189077167598475


In [0]:
print(f"r2: {trainingSummary.r2}")

r2: 0.022861466913958184


In [0]:
# Generalized Linear Regression
from pyspark.ml.regression import GeneralizedLinearRegression


In [0]:
glr=GeneralizedLinearRegression(labelCol="label",featuresCol="features", family="gaussian",link="identity",maxIter=10,regParam=0.3)


model=glr.fit(training)

In [0]:
model.coefficients

Out[23]: DenseVector([0.0105, 0.8003, -0.7845, 2.368, 0.501, 1.1222, -0.2927, -0.4984, -0.6036, 0.6726])

In [0]:
model.intercept

Out[24]: 0.14592176145232041

In [0]:
summary=model.summary

In [0]:
print(f"Coeficient Standard Errors: {summary.coefficientStandardErrors}")

print(f"T Values: {summary.tValues}")

print(f"P Values: {summary.pValues}")

print(f"Dispersion: {summary.dispersion}")

print(f"Null Deviance: {summary.nullDeviance}")

print(f"Residual Degree of Freedom Null: {summary.residualDegreeOfFreedomNull}")

print(f"Deviance: {summary.deviance}")

print(f"Residual Degree of Freedom: {summary.residualDegreeOfFreedom}")

print(f"AIC {summary.aic}")

print(f"Deviance Residuals: {summary.residuals().show()}")


Coeficient Standard Errors: [0.7950428434287478, 0.8049713176546897, 0.7975916824772489, 0.8312649247659919, 0.7945436200517938, 0.8118992572197593, 0.7919506385542777, 0.7973378214726764, 0.8300714999626418, 0.7771333489686802, 0.463930109648428]
T Values: [0.013259446542269243, 0.9942283563442594, -0.9836067393599172, 2.848657084633759, 0.6305509179635714, 1.382234441029355, -0.3695715687490668, -0.6250446546128238, -0.7271418403049983, 0.8654306337661122, 0.31453393176593286]
P Values: [0.989426199114056, 0.32060241580811044, 0.3257943227369877, 0.004575078538306521, 0.5286281628105467, 0.16752945248679119, 0.7118614002322872, 0.5322327097421431, 0.467486325282384, 0.3872259825794293, 0.753249430501097]
Dispersion: 105.60988356821714
Null Deviance: 53229.3654338832
Residual Degree of Freedom Null: 500
Deviance: 51748.8429484264
Residual Degree of Freedom: 490
AIC 3769.1895871765314
+-------------------+
|  devianceResiduals|
+-------------------+
|-10.974359174246889|
| 0.8872320138

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
data=spark.read.format("libsvm").load("file:///databricks/driver/spark/data/mllib/sample_libsvm_data.txt")

In [0]:
data.show(truncate=False,n=4)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
featureIndexer=VectorIndexer(maxCategories=4,inputCol="features",outputCol="indexedFeatures").fit(dataset=data)

In [0]:
trainingData,testData=data.randomSplit(weights=(0.7,0.3),seed=101)

In [0]:
dt=DecisionTreeRegressor(featuresCol="indexedFeatures")

In [0]:
pipeline=Pipeline(stages=(featureIndexer,dt,))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions=model.transform(dataset=testData)

In [0]:
predictions.show(truncate=False,n=4)

+-----+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [0]:
evaluator=RegressionEvaluator(labelCol="label",predictionCol="prediction",metricName="rmse")

In [0]:
rmse=evaluator.evaluate(dataset=predictions)

print(f"Root Mean Squared Error : {rmse}")

Root Mean Squared Error : 0.0


In [0]:
# Summary:

model.stages[1]

Out[26]: DecisionTreeRegressionModel: uid=DecisionTreeRegressor_2858580476de, depth=2, numNodes=5, numFeatures=692

In [0]:
# Random Forest Regression

from pyspark.ml.regression import RandomForestRegressor

In [0]:
rf=RandomForestRegressor(featuresCol="indexedFeatures")

In [0]:
pipeline=Pipeline(stages=(featureIndexer,rf,))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions=model.transform(dataset=testData)

In [0]:
rmse=evaluator.evaluate(dataset=predictions)

print(f"Root Mean Squared Error : {rmse}")

Root Mean Squared Error : 0.0570087712549569


In [0]:
# Summary:

model.stages[1]

Out[34]: RandomForestRegressionModel: uid=RandomForestRegressor_56f88eeae2a4, numTrees=20, numFeatures=692

In [0]:
# Gradient Boosted Tree Regression

from pyspark.ml.regression import GBTRegressor

In [0]:
gbt=GBTRegressor(featuresCol="indexedFeatures", maxIter=10)

In [0]:
pipeline=Pipeline(stages=[featureIndexer,gbt])

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions=model.transform(dataset=testData)

In [0]:
rmse=evaluator.evaluate(dataset=predictions)
print(f"Root Mean Squared Error : {rmse}")

Root Mean Squared Error : 0.0


In [0]:
# summary

model.stages[1]

Out[42]: GBTRegressionModel: uid=GBTRegressor_b6ef5bda5e9c, numTrees=10, numFeatures=692

In [0]:
# Factorizations Machines Regressor

from pyspark.ml.regression import FMRegressor
from pyspark.ml.feature import MinMaxScaler

In [0]:
featureScaler=MinMaxScaler(inputCol="features",outputCol="scaledFeatures").fit(trainingData)

In [0]:
fm=FMRegressor(featuresCol="scaledFeatures",labelCol="label",stepSize=0.001)

In [0]:
pipeline=Pipeline(stages=(featureScaler,fm,))

In [0]:
model=pipeline.fit(dataset=trainingData)

In [0]:
predictions=model.transform(dataset=testData)

In [0]:
rmse=evaluator.evaluate(dataset=predictions)

print(f"Root Mean Squared Error : {rmse}")

Root Mean Squared Error : 0.08249159337841676
